# "Process data with Ray"
> "Awesome summary"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [fastpages, jupyter]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [ ]:
!pip install -U ray

     |████████████████████████████████| 54.7 MB 30 kB/s 
     |████████████████████████████████| 119 kB 54.1 MB/s 


In [ ]:
pip install pandas-datareader==0.10.0

     |████████████████████████████████| 109 kB 37.8 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader as web
import itertools

In [ ]:
import ray

# start ray
ray.init(num_cpus=6)


{'metrics_export_port': 62438,
 'node_id': '01d331c749e42aab051c37dc324fa73540f4da1c2c936884e451fc76',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-11-22_19-09-20_711468_62/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-11-22_19-09-20_711468_62/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-11-22_19-09-20_711468_62',
 'webui_url': None}

In [ ]:
def get_data(ticker, dt_init, dt_end):
    return web.get_data_yahoo(ticker+'.SA', start=dt_init, end=dt_end)


@ray.remote
def get_data_remote(ticker, dt_init, dt_end):
    return web.get_data_yahoo(ticker+'.SA', start=dt_init, end=dt_end)


def processa_data(x):
    return x['Adj Close'].pct_change()

@ray.remote
def processa_data_remote(x):
    return x['Adj Close'].pct_change()


def ingest(ticker, dt_init, dt_end):
    df = get_data(ticker, dt_init, dt_end)
    df['retorno'] = processa_data(df)
    return df.dropna()

@ray.remote
def ingest_remote(ticker, dt_init, dt_end):
    df = get_data(ticker, dt_init, dt_end)
    df['retorno'] = processa_data(df)
    return df.dropna()




In [ ]:

# benchmarking
tickers = ['ELET3', 'ELET6','PETR3', 'MGLU3', 'ABEV3', 'ALPA4', 'BRFS3', 'GOLL4', 'HYPE3', 'RENT3', 'MRFG3', 'RADL3', 'SUZB3', 'VALE3']

data_init = '2010-01-01'
data_end = '2021-09-30'



In [ ]:
%%time
resultado = [get_data(ticker, data_init, data_end) for ticker in tickers]

CPU times: user 1.51 s, sys: 147 ms, total: 1.65 s
Wall time: 11.8 s


In [ ]:
%%time 
resultado_ray = [get_data_remote.remote(ticker, data_init, data_end) for ticker in tickers]
resultado_ray = ray.get(resultado_ray)

CPU times: user 191 ms, sys: 40.1 ms, total: 231 ms
Wall time: 5.44 s


In [ ]:
resultado_ray

In [ ]:
%%time
resultado_processamento = [processa_data(x) for x in resultado]

CPU times: user 16.8 ms, sys: 0 ns, total: 16.8 ms
Wall time: 21.4 ms


In [ ]:
resultado_processamento

In [ ]:
%%time
resultado_processamento_remote = [processa_data_remote.remote(x) for x in resultado_ray]
resultado_processamento_remote = ray.get(resultado_processamento_remote)

CPU times: user 40.6 ms, sys: 18.4 ms, total: 58.9 ms
Wall time: 102 ms


In [ ]:
%%time
dados = [ingest(ticker, data_init, data_end) for ticker in tickers]

CPU times: user 1.89 s, sys: 178 ms, total: 2.07 s
Wall time: 32.2 s


In [ ]:
dados

In [ ]:
%%time 
dados_ray = [ingest_remote.remote(ticker, data_init, data_end) for ticker in tickers]
dados_ray = ray.get(dados_ray)

CPU times: user 187 ms, sys: 40.9 ms, total: 228 ms
Wall time: 7.35 s


In [ ]:
dados_ray

In [ ]:
combinacoes = list(itertools.combinations(dados[0].columns, 3))

In [ ]:
def statistics(df, combinacoes):
    return df[combinacoes].resample('Q').pad().describe()

@ray.remote
def statistics_remote(df, combinacoes):
    return df[combinacoes].resample('Q').pad().describe()


In [ ]:
%%time
stats = [statistics(df, list(combinacao)) for combinacao in combinacoes for df in dados]

CPU times: user 5.76 s, sys: 249 ms, total: 6.01 s
Wall time: 6.06 s


In [ ]:
%%time
stats_ray = ray.get([statistics_remote.remote(df, list(combinacao)) for combinacao in combinacoes for df in dados])

(statistics_remote pid=2113) 
CPU times: user 1.88 s, sys: 481 ms, total: 2.36 s
Wall time: 8.36 s


In [ ]:
combinacoes

[('High', 'Low', 'Open'),
 ('High', 'Low', 'Close'),
 ('High', 'Low', 'Volume'),
 ('High', 'Low', 'Adj Close'),
 ('High', 'Low', 'retorno'),
 ('High', 'Open', 'Close'),
 ('High', 'Open', 'Volume'),
 ('High', 'Open', 'Adj Close'),
 ('High', 'Open', 'retorno'),
 ('High', 'Close', 'Volume'),
 ('High', 'Close', 'Adj Close'),
 ('High', 'Close', 'retorno'),
 ('High', 'Volume', 'Adj Close'),
 ('High', 'Volume', 'retorno'),
 ('High', 'Adj Close', 'retorno'),
 ('Low', 'Open', 'Close'),
 ('Low', 'Open', 'Volume'),
 ('Low', 'Open', 'Adj Close'),
 ('Low', 'Open', 'retorno'),
 ('Low', 'Close', 'Volume'),
 ('Low', 'Close', 'Adj Close'),
 ('Low', 'Close', 'retorno'),
 ('Low', 'Volume', 'Adj Close'),
 ('Low', 'Volume', 'retorno'),
 ('Low', 'Adj Close', 'retorno'),
 ('Open', 'Close', 'Volume'),
 ('Open', 'Close', 'Adj Close'),
 ('Open', 'Close', 'retorno'),
 ('Open', 'Volume', 'Adj Close'),
 ('Open', 'Volume', 'retorno'),
 ('Open', 'Adj Close', 'retorno'),
 ('Close', 'Volume', 'Adj Close'),
 ('Close', 